In [1]:
import fitz  # PyMuPDF
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


In [2]:
# 1. Ler Pdf
def read_pdf_text(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

In [3]:
# 2. Dividir o texto
def split_text(text, chunk_size=1000, chunk_overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )
    return text_splitter.split_text(text)

In [4]:
# 3. LLM
llm = Ollama(model="mistral", temperature=0.0)

C:\Users\Prady\AppData\Local\Temp\ipykernel_12352\1385775398.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral", temperature=0.0)


In [19]:
# 4. Prompt
from langchain_ollama import OllamaLLM

prompt = PromptTemplate(
    input_variables=["chunk"],
    template = """
                You must extract all IP addresses from the following text:

                {chunk}
                Rules:
                1. Return ONLY the list of IP addresses.
                2. Each IP must be on a separate line.
                3. Do NOT include any explanation or formatting.
    
            """
)

chain = prompt | llm


In [30]:
import re

def extract_valid_ips(text):
    print(f"Processing chunk of size: {len(text)}")
    print(f"Text: {text[:100]}...")  # Print the first 100 characters for debugging
    
    return re.findall(r'\b(?:\d{1,3}\.){3}\d{1,3}\b', text)

In [ ]:
# 5. Juntar tudo

def extract_ips_from_pdf(pdf_path: str) -> list:
    """
    Extrai os IPs de um PDF:
      1. Lê o texto do PDF.
      2. Divide o texto em chunks.
      3. Executa a cadeia de LLM em cada chunk para extrair os IPs.
      4. Usa expressões regulares para filtrar IPs válidos.
    Retorna uma lista com todos os IPs extraídos.
    """
    text = read_pdf_text(pdf_path)
    chunks = split_text(text)
    ips = []
    for chunk in chunks:
        result = chain.invoke({"chunk": chunk})
        ips += extract_valid_ips(result)
    return ips


In [ ]:
caminho_pdf = "modelo operadora3.pdf"
ips_encontrados = extract_ips_from_pdf(caminho_pdf)
print("\nIPs extraídos:\n")
print(ips_encontrados)

KeyboardInterrupt: 

In [32]:
raw_text = ips_encontrados

# Regex for valid IPv4 addresses
ipv4_pattern = r'\b(?:\d{1,3}\.){3}\d{1,3}\b'

# Regex for valid IPv6 addresses (simple version)
ipv6_pattern = r'\b(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}\b'

# Find all matches
ipv4_list = re.findall(ipv4_pattern, raw_text)
ipv6_list = re.findall(ipv6_pattern, raw_text)

# Remove duplicates and sort
unique_ipv4 = sorted(set(ipv4_list))
unique_ipv6 = sorted(set(ipv6_list))

# Output
print("IPv4 Addresses:")
print(unique_ipv4)

print("\nIPv6 Addresses:")
print(unique_ipv6)

IPv4 Addresses:
['111.11.111.1', '111.11.111.11', '111.11.111.111']

IPv6 Addresses:
['1111:11d:1e11:11bc:bc11:1111:11ba:4e11']


In [33]:
# Get the filename without extension
base_name = os.path.splitext(caminho_pdf)[0]
# Combina as duas listas e remove duplicatas
combined_ips = list(set(ipv4_list + ipv6_list))

# Gera o nome do arquivo de saída baseado no nome do PDF
base_name = os.path.splitext(caminho_pdf)[0]
txt_filename = f"{base_name}_ips.txt"

# Salva os IPs no arquivo
with open(txt_filename, "w") as f:
    for ip in combined_ips:
        f.write(f"{ip}\n")

print(f"✅ IPs combinados e salvos em: {txt_filename}")

✅ IPs combinados e salvos em: modelo operadora3_ips.txt


In [34]:
combined_ips

['1111:11d:1e11:11bc:bc11:1111:11ba:4e11',
 '111.11.111.111',
 '111.11.111.1',
 '111.11.111.11']

In [2]:
import requests
import pandas as pd

# Lista para armazenar os dados de cada IP
ips = ["111.11.111.111"]
# ips = ["8.8.8.8", "1.1.1.1"]


dados = ips

# Loop pelos IPs
for ip in ips:
    try:
        url = f"http://ip-api.com/json/{ip}?fields=status,proxy,mobile,country,city,hosting,org,regionName"
        response = requests.get(url)
        info = response.json()

        if info['status'] == 'success':
            dados.append({
                "ip": ip,
                "proxy": info.get("proxy"),
                "mobile": info.get("mobile"),
                "country": info.get("country"),
                "city": info.get("city"),
                "hosting": info.get("hosting"),
                "org": info.get("org"),
                "regionName": info.get("regionName")
            })
        else:
            print(f"❌ Falha ao consultar {ip}: {info.get('message', 'erro desconhecido')}")

    except Exception as e:
        print(f"⚠️ Erro ao consultar {ip}: {e}")

# Cria um DataFrame com os resultados
df = pd.DataFrame(dados)

# Salva em CSV
output_csv = "resultado_consulta_ips.csv"
df.to_csv(output_csv, index=False)

print(f"\n✅ Consulta finalizada! Resultado salvo em: {output_csv}")


❌ Falha ao consultar {'ip': '111.11.111.111', 'proxy': False, 'mobile': True, 'country': 'China', 'city': 'Shijiazhuang', 'hosting': False, 'org': 'China Mobile', 'regionName': 'Hebei'}: erro desconhecido

✅ Consulta finalizada! Resultado salvo em: resultado_consulta_ips.csv


In [3]:
import requests

def checar_ip_virustotal(ip, api_key):
    url = f"https://www.virustotal.com/api/v3/ip_addresses/{ip}"
    headers = {
        "x-apikey": api_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        stats = data['data']['attributes']['last_analysis_stats']
        malicioso = stats.get("malicious", 0)
        suspeito = stats.get("suspicious", 0)

        print(f"IP: {ip}")
        print(f"Malicioso: {malicioso}")
        print(f"Suspeito: {suspeito}")
        return stats
    else:
        print(f"Erro na requisição: {response.status_code}")
        return None

# Exemplo de uso:
API_KEY = "0599ef2145a358f649a363038cf91418a51934b4f95e4c0ce06a49060c3086ca"
checar_ip_virustotal("111.11.111.111", API_KEY)


IP: 111.11.111.111
Malicioso: 0
Suspeito: 0


{'malicious': 0,
 'suspicious': 0,
 'undetected': 33,
 'harmless': 61,
 'timeout': 0}

In [14]:
326+90-(110+30+17+90)

169

In [37]:
combined_ips

['1111:11d:1e11:11bc:bc11:1111:11ba:4e11',
 '111.11.111.111',
 '111.11.111.1',
 '111.11.111.11']

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

# Defina aqui sua lista de IPs diretamente
lista_ips = combined_ips

# Lista para armazenar os resultados
resultados = []

# Iniciar o navegador (apenas uma vez)
navegador = webdriver.Chrome()
navegador.get('https://registro.br/tecnologia/ferramentas/whois?search=')
navegador.maximize_window()

for ip in lista_ips:
    try:
        # Localizar a caixa de pesquisa
        box = navegador.find_element(By.ID, 'whois-field')

        # Clicar e limpar o campo de pesquisa
        box.click()
        box.clear()
        time.sleep(1)

        # Digitar o IP e pressionar Enter
        box.send_keys(ip)
        time.sleep(2)
        box.send_keys(Keys.RETURN)
        time.sleep(3)  # Esperar a página carregar

        # Capturar o dono do IP
        try:
            dono = navegador.find_element(By.CLASS_NAME, 'cell-owner')  # Verifique se essa classe está correta
            texto_extraido = dono.text.strip()

            # Se o texto extraído estiver vazio, definir como "Nenhuma informação encontrada"
            if not texto_extraido:
                texto_extraido = "Nenhuma informação encontrada"
        except Exception:
            texto_extraido = "Nenhuma informação encontrada"

        # Adicionar resultado à lista
        resultados.append({"IP": ip, "Dono": texto_extraido})

    except Exception as e:
        print(f"Erro ao pesquisar {ip}: {e}")

# Fechar o navegador (apenas no final)
navegador.quit()

# Salvar os resultados em uma planilha
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("Resultados_WHOIS.xlsx", index=False)

print("Processo concluído! Resultados salvos em 'Resultados_WHOIS.xlsx'")


Processo concluído! Resultados salvos em 'Resultados_WHOIS.xlsx'
